# Context
This notebook drives the training process for different models.

In [1]:
# Set project's environment variables
import os
import sys
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../../project.env")
sys.path.append(os.environ["PYTHONPATH"])

In [2]:
# Import project-wide and PH2 specific variables and functions
import superheader as sup
import TRAIN.architecture.BERT.bert as bert

# Models

## Setup

In [3]:
import torch
import gc

In [4]:
TRAIN_classes = 'all-classes'
exploringBERT = bert.BERT_MINI
exploring_batch_size = 1024

## Train

In [5]:
sup.bert_score_tracker = []

In [ ]:
for data_unit in [sup.DATA_S_PF, sup.DATA_S_PV]:
  bert.find_best(data_unit=data_unit, 
                  label_col=sup.class_numeric_column,
                  class_list=TRAIN_classes, 
                  batch_size=exploring_batch_size,
                  LOADABLE_CANDIDATES=[exploringBERT],
                  )
  
  gc.collect()
  if torch.backends.mps.is_available():
    torch.mps.empty_cache()

gc.collect()
if torch.backends.mps.is_available():
  torch.mps.empty_cache()

{'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'all-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 15}
{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-mini-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 3996}


0.8905265406386842
updating best... at 2025-06-22_21:21:23



{'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'all-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': True, 'PH3': False, 'reducer': '', 'kernel': '', 'n': 75}
{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-mini-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 3996}


0.9201110755099862
updating best... at 2025-06-22_22:27:37



{'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'all-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': False, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 15}
{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-mini-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 3996}


0.9530065150058742
updating best... at 2025-06-22_23:33:24



{'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'all-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': False, 'PH3': False, 'reducer': '', 'kernel': '', 'n': 72}
{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-mini-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 3996}


0.9665705436291787
updating best... at 2025-06-23_00:39:56



Data Unit: Spf
Best score: 0.9665705436291787
Best data config: {'data_unit': 'Spf', 'label_col': 'class_numeric', 'class_list': 'all-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': False, 'PH3': False, 'reducer': '', 'kernel': '', 'n': 72}
Best train config: {'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-mini-uncased', 'optimizer': <class 'torch.optim.adamw.AdamW'>, 'lr': 1e-05, 'weight_decay': 0, 'loss_fn': <class 'torch.nn.modules.loss.CrossEntropyLoss'>, 'num_epochs': 3996}
{'data_unit': 'Spv', 'label_col': 'class_numeric', 'class_list': 'all-classes', 'num_classes': None, 'difficulty': None, 'class_numeric_list': None, 'batch_size': 1024, 'PH2': True, 'PH3': True, 'reducer': 'pca', 'kernel': '', 'n': 15}
{'arch': 'BERT', 'device': device(type='mps'), 'loadable': 'gaunernst/bert-mini-uncased', 'optimizer': <class 'torch.optim.adamw.Adam

Training:  17%|█▋        | 2667/15984 [24:51<2:06:34,  1.75epoch/s, epoch=2667, loss=3.69]

# Keep metrics

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
best_scores_df = pd.DataFrame(sup.bert_score_tracker, columns=sup.bert_scores_columns)
now = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
sup.create_dir_if_not_exists(os.path.join(sup.TRAIN_SCORES_ROOT, TRAIN_classes, 
                                          sup.TRAIN_BERT_CODE, exploringBERT))
best_scores_df.to_csv(os.path.join(sup.TRAIN_SCORES_ROOT, TRAIN_classes,
                                       sup.TRAIN_BERT_CODE, exploringBERT,
                                       f"BERTbest-{now}.csv"), index=False)